In [72]:
from rectools import Columns
import pandas as pd
from rectools.dataset import Dataset
from rectools.models import PopularModel

interactions_df = pd.read_csv('../data/interactions.csv')

interactions_df.rename(
    columns={
        'last_watch_dt': Columns.Datetime,
        'total_dur': Columns.Weight
    }, inplace=True)

pop_model = PopularModel()

pop_model.fit(Dataset.construct(interactions_df))

popular_100 = pop_model.recommend(
    users=[1],
    dataset=Dataset.construct(interactions_df),
    k=100,
    filter_viewed=False,
)

In [73]:
popular_100

,user_id,item_id,score,rank
0,1,10440,202457.0,1
1,1,15297,193123.0,2
2,1,9728,132865.0,3
3,1,13865,122119.0,4
4,1,4151,91167.0,5
...,...,...,...,...
95,1,11754,6882.0,96
96,1,10761,6877.0,97
97,1,12356,6874.0,98
98,1,8447,6762.0,99


In [74]:
popular_100.item_id.to_json('../saved_models/popular.json')

In [75]:
from saved_models.models import load_model_from_pickle
als_ann = load_model_from_pickle("../saved_models/als_ann.pkl")
popular = pd.read_json('../saved_models/popular.json', orient='index')

In [77]:
popular[0].values.tolist()[:10]

[10440, 15297, 9728, 13865, 4151, 3734, 2657, 4880, 142, 6809]

In [79]:
from types import MethodType


def recommend_als_ann(self, user_id, N_recs=10, popular=popular):
    if user_id in self.user_id_map.external_ids:
        return self.get_item_list_for_user(user_id, N_recs).tolist()
    else:
        return popular[0].values.tolist()[:10]


als_ann.recommend = MethodType(recommend_als_ann, als_ann)

In [83]:
als_ann.recommend(user_id=3)

[3784, 4880, 7793, 12192, 2657, 7829, 14899, 7417, 3935, 3734]